<a href="https://colab.research.google.com/github/alexiamhe93/RAMP_method/blob/main/RAMP%20Python%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for replicating the Repeated Adjustment of Measurement Protocols (RAMP) method case study

This Python notebook is used to replicate the results of the paper titled:

> "Repeated Adjustment of Measurement Protocols (RAMP) method for developing high-validity text classifiers"

The notebook provides the code for replicating the three different stages of RAMP (manual coding, classifier development, and integrative evaluation) and compares the RAMP method against a non-iterative approach.

The notebook was designed using Google Colab on an Nvidia T4 GPU (free with log-in). The code works locally but all dependencies from the "Load packages" will have to be installed.




# Table of contents:

>[Code for replicating the Repeated Adjustment of Measurement Protocols (RAMP) method case study](#scrollTo=Ht0MhYw8ggZq)

>[Table of contents:](#scrollTo=7x5QO1Ata5Pb)

>[Initiate notebook](#scrollTo=0NK-mCzVj-K0)

>>[0.1 Install and load packages](#scrollTo=l1uWzkOxkBaG)

>>[0.2 Download data and pre-trained BERT model](#scrollTo=9NbRD_KokEAA)

>>[0.3 Functions used across stages](#scrollTo=6gRWX3kBIlkj)

>[Manual Coding (RAMP method)](#scrollTo=7WBrqsVvvXiL)

>>[1.1 Input](#scrollTo=6Q0QqaihHSSW)

>>[1.2 Throughput](#scrollTo=Ci0ngC2BHryG)

>>[1.3 Output](#scrollTo=twg1bGTWJjzF)

>>>[1.3.1 Sense check manual coding output](#scrollTo=3ZBProBJJ2-m)

>[Classifier development (RAMP method)](#scrollTo=JqxQnDCpvdG_)

>>[2.1 Input](#scrollTo=jxIBD7-LvgLG)

>>[2.2 Throughput](#scrollTo=W-K1ar4fvkHL)

>>>[2.2.1 Training and developing classifiers](#scrollTo=RUgfO-RPvoYR)

>>>>[Rule-based development](#scrollTo=iyoKT7OXv01M)

>>>>[Supervised classifier training](#scrollTo=qMMmlFBbv-2r)

>>>>[LLM development](#scrollTo=FAcWi6TZwBl8)

>>>[2.2.2 Results](#scrollTo=_7Vgu5ejwDgP)

>>[2.3 Output](#scrollTo=rLGzncfnSdhu)

>>>[2.3.1 Rule-based classifier](#scrollTo=IXqrWohnSvde)

>>>[2.3.2 Supervised classifier](#scrollTo=gXBsDJ9zS3DS)

>>>[2.3.3 LLM classifier](#scrollTo=lVMM_HDmS85T)

>[Integrative evaluation (RAMP method)](#scrollTo=5QvklA8-nEJT)

>>[3.1 Inter-rater discrepancies](#scrollTo=j4wwlA44mblb)

>>[3.2 Supervised misclassifications](#scrollTo=Ym2-GFNumkRr)

>>>[3.2.1 Using LIME to explore BERT classifier](#scrollTo=C6djwU6rmv6K)

>[Non-iterative method results](#scrollTo=hf28QA-VLCpI)

>>[4.1 Manual coding](#scrollTo=bl9ZsGWlLIL-)

>>>[4.1.1 Compare RAMP and non-iterative manual coding](#scrollTo=e8F_rO_6YVJB)

>>[4.2 Classifier development](#scrollTo=vTckTVXjMPh8)

>>>[4.2.1 Training and developing classifiers](#scrollTo=BHcGRbqhMSI2)

>>>[4.2.2 Output](#scrollTo=DneDbUf3M5NB)

>[Conclusions](#scrollTo=VbMaI0IKk_0H)

>[References](#scrollTo=bulsXEAaP83Z)



# 0. Initiate notebook

This section installs all the necessary Python packages to complete these analysis. We also download the data and pre-trained BERT model for replicating the results.


## 0.1 Install and load packages

A few packages require mention as they are non-standard:

1. **spacy** ([Honnibal et al., 2022](https://github.com/explosion/spaCy)) This package is used for creating a rule-based dictionary classifier, similar to LIWC ([Pennebaker et al., 2001](http://downloads.liwc.net.s3.amazonaws.com/LIWC2015_OperatorManual.pdf)). This

2. **ktrain** ([Maiya, 2022](https://github.com/amaiya/ktrain)): This package is a Keras wrapper for streamlining many tasks related to fine-tuning and deploying deep learning models. In this notebook we use it to fine-tune Google's BERT ([Devlin et al., 2019](https://arxiv.org/abs/1810.04805)) base model.

3. **eli5** ([Korobov, 2017](https://av.tib.eu/media/33771);[Korobov & Lopuhin, 2024](https://github.com/eli5-org/eli5)): "Explain like I'm five" is a package used for running the LIME ([Ribeiro et al., 2016](http://arxiv.org/abs/1602.04938)) algorithm to examine how a supervised classifier is making its predictions.

4. **openai** ([OpenAI et al., 2024](https://platform.openai.com/docs/api-reference/introduction
 )): This package accesses the OpenAI API for using GPT-4o within the notebook.

5. **textstat** ([Shivan & Chaitanya, 2024](https://pypi.org/project/textstat/)): This package calculates simple statistical information relating to raw text data.


In [ ]:
# ktrain relies on the tf-keras: keras with added tensorflow functionality
!pip install tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
# For Supervised classifier
!pip install ktrain
# For revealing under the classifier black box
!pip install https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip
# For LLM classifier
!pip install openai
# For summary statistics
!pip install textstat
# This is a port from Gwet's R package with the same name
!pip install irrCAC

In [ ]:
# General use packages
import requests, zipfile, io, os, psutil, random, time
import torch
import pandas as pd
# This deactivates a warning from Pandas that frequently prints
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from collections import Counter
from tqdm import tqdm
# For descriptive statistics
from textstat.textstat import textstatistics
import re
# Performance evaluations for binary classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc, roc_auc_score, matthews_corrcoef
# For calculating inter-rater reliability
from irrCAC.raw import CAC
#for troubleshooting
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from nltk import agreement
import matplotlib.pyplot as plt

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# for rule-based classification
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")

In [ ]:
# for supervised classification
import ktrain
from ktrain import text
# for LLM classification
import openai

In [ ]:
# Check system GPU (recommended if possible)
# CPU cores
num_cpu_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cpu_cores}")
# GPU details
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # in GB
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU available.")

Number of CPU cores: 2
No GPU available.


In [ ]:
# Load in openai keys for few-shot classifier
oai_k = "your-API-key-here"
openai.organization = "your-organization-key-here" #if applicable
openai.api_key = oai_k
os.environ['OPENAI_API_KEY'] = oai_k

## 0.2 Download data and pre-trained BERT model

All the data for replication (<50mb) is accessed through a GitHub link and the pre-trained BERT model (1.03GB) from dropbox

Download data from GitHub

In [ ]:
# Download empirical data
r = requests.get('https://github.com/alexiamhe93/RAMP_method/blob/main/Dataset/data.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# Load data for use across the notebook
try:
  # Training + Validation data (70%) and Test data (30%) for RAMP and non-iterative
  train_test = pd.read_csv("data/train_test_data.csv")
  # Throughput results for Manual Coding (1st stage)
  St1Through = pd.read_csv("data/RAMP_Stage1.csv")
  # Throughput results for Computation (1st stage)
  St2Through = pd.read_csv("data/RAMP_Stage2.csv")
  # Output results for 1st and 2nd Stage of RAMP / Input for Evaluation (3rd stage)
  out = pd.read_csv("data/RAMP_Stage3.csv")
  # Results of the non-iterative method for validating RAMP
  out_NonIterative = pd.read_csv("data/results_OneShot.csv")
except:
  # Sometimes the unzip puts the csvs in the directory rather than a data folder
  train_test = pd.read_csv("train_test_data.csv")
  St1Through = pd.read_csv("RAMP_Stage1.csv")
  St2Through = pd.read_csv("RAMP_Stage2.csv")
  out = pd.read_csv("RAMP_Stage3.csv")
  out_NonIterative = pd.read_csv("results_OneShot.csv")

Split training and test data for RAMP method and non-iterative method.

Although the two datasets are the same texts, the train/test split is different as they were stratified for the distribution of Misunderstandings. The non-iterative data has fewer misunderstandings so a new stratification was necessary.

In [ ]:
# Train used for training and validation data for developing the classifiers
train = train_test[train_test["train_test"] == "train"][["text", "Misunderstanding"]]
# Test withheld for validation - outputs for final classifiers on test data in "out" dataframe
test = train_test[train_test["train_test"] == "test"][["text", "Misunderstanding"]]

# Training / validation data for the non-iterative method
train_NonIterative = train_test[train_test["train_test_OneShot"] == "train"][["text", "Misunderstanding_OneShot"]]
# Test data for the non-iterative method
test_NonIterative = train_test[train_test["train_test_OneShot"] == "test"][["text", "Misunderstanding_OneShot"]]

Download RAMP supervised classifier model from Dropbox (~1GB). This can take some time if internet is slow .

In [ ]:
!wget -O supervised_model.zip https://www.dropbox.com/scl/fi/5wtuor1ag1gktg6eukqwr/supervised_model.zip?rlkey=nfwxyataobjzt708m3gf27o3s&st=cz5r9lq0&dl=0 --quiet
!unzip supervised_model.zip

## 0.3 Functions used across stages

In [ ]:
class TextDataStats:
  def __init__(self, df, text_column="text", binary_column="Misunderstanding",
               IRR_columns = ["Coder1","Coder2","Coder3","Coder4"],
               group_column = "Round"):
    self.df = df
    self.text_column = text_column
    self.binary_column = binary_column
    self.IRR_columns = IRR_columns
    self.group_column = group_column

  def preprocess_text(self):
    """
    Extracts words and sentences from the text, counts them and adds to the dataframe.
    """
    self.df['words'] = self.df[self.text_column].apply(lambda x: re.findall(r'\b\w+\b', x.lower()))
    self.df['word_count'] = self.df['words'].apply(len)

  def basic_stats(self):
    """
    Computes basic statistics for overall and grouped data.
    """
    self.preprocess_text()

    # General stats
    general_stats = self.df.describe(include=[np.number]).loc[['mean', 'std', 'min', '50%', 'max'], ['word_count']]
    general_stats.rename(index={'50%': 'median'}, inplace=True)
    # Grouped stats by binary column
    grouped_stats = self.df.groupby(self.binary_column).agg({
        'word_count': ['mean', 'median', 'std', 'min', 'max'],
    })
    # Binary column distribution
    binary_dist = self.df[self.binary_column].value_counts(normalize=True).to_frame('distribution')
    return general_stats.round(2), grouped_stats.round(2), binary_dist.round(2)

  def BasicReport(self):
    """
    Generates a report combining all statistics in a readable text format.
    """
    general_stats, grouped_stats, binary_dist = self.basic_stats()

    # Creating a structured text report
    report = "Text Data Statistics Report\n\n"
    report += "General Statistics:\n"
    report += general_stats.to_string() + "\n\n"

    report += "Statistics by Binary Column:\n"
    for name, group in self.df.groupby(self.binary_column):
        report += f"\nGroup: {name}\n"
        report += grouped_stats.loc[name].to_string() + "\n"
    return report


  def get_IRR(self, df):
    """
    Fetches the absolute agreement, Krippendorff's Alpha, Gwet's AC1
    """
    df = df[self.IRR_columns]
    # Get absolute agreement
    df = df.astype(int)
    IRR_out = []
    for i, row in df.iterrows():
      for k in list(df.columns):
        IRR_out.append([k, str(i), row[k]])
    ratingtask = agreement.AnnotationTask(data=IRR_out)
    ags = ratingtask.avg_Ao()
    # Get Gwet AC1 and K Alpha
    cac= CAC(df)
    print(cac)
    #print(cac_4raters)
    Gwet_obj = cac.gwet()
    Alpha_obj = cac.krippendorff()
    return ags, Alpha_obj, Gwet_obj

  def process_object(self, IRR_obj, sig_level = 0.001):
    """
    This processes the cac output for the IRR into two strings for reporting
    """
    s = IRR_obj["est"]["coefficient_value"]
    ci1 = IRR_obj["est"]["confidence_interval"][0]
    ci2 = IRR_obj["est"]["confidence_interval"][1]
    stat_string = f"{s:.2f} CI = ({ci1:.2f}, {ci2:.2f})"
    Z = IRR_obj["est"]["z"]
    pval = IRR_obj["est"]["p_value"]
    if pval < sig_level:
      sig_string = f"z = {Z:.2f}; p < {sig_level}"
    else:
      sig_string = f"z = {Z:.2f}; p = {pval:.3f}"

    return stat_string, sig_string


  def IRRreport(self):
    """
    Produces the inter-rater reliability report
    """
    df = self.df.sort_values([self.group_column])
    rounds = df[self.group_column].unique()
    agreement, alphas_, alpha_sigs_ = [],[],[]
    ac1s, ac1s_sigs, ss = [],[],[]
    for i in rounds:
      tdf = df[df[self.group_column] == i]
      ss.append(len(tdf))
      ags, alpha_, gwets_ = self.get_IRR(tdf)
      agreement.append(ags)
      alpha_stat, alpha_sig = self.process_object(alpha_)
      alphas_.append(alpha_stat)
      alpha_sigs_.append(alpha_sig)

      ac1_stat, ac1_sig = self.process_object(gwets_)
      ac1s.append(ac1_stat)
      ac1s_sigs.append(ac1_sig)

    return pd.DataFrame({"Round":["Round " + str(i) for i in rounds],
                         "Sample size":ss, "Agreement":agreement,
                         "K's Alpha":alphas_,"K's Alpha significance":alpha_sigs_,
                         "Gwet's AC1":ac1s,"Gwet's AC1 significance":ac1s_sigs,
                         })

  def get_disagreements(self,n=10, return_df = False):
    """
    Prints n disagreements for the IRR results
    """
    df = self.df
    disag = []
    for i, row in df.iterrows():
      x = 0
      for coder in self.IRR_columns:
        x += row[coder]
      disag.append(x)
    df["disag"] = disag
    ncoders = len(self.IRR_columns)
    df = df[df["disag"] < ncoders]
    df = df[df["disag"] > 0]
    if return_df:
      return df.round(2)
    else:
      sdf = df.sample(n)
      for s in sdf.text:
        print("----------")
        print(s)

  def get_misclassifications(self, n=5, return_all=False):
    """
    Function to report on the misclassifications across all three classifiers.
    """
    df = self.df

    def classify(row):
      base, fs, sup = int(row["Manual"]), int(row["LLM"]), int(row["supervised"])
      if sup == base:
        return "TP (All)" if base == 1 else "TN (All)" if fs == base else "FP (LLM)" if base == 0 else "FN (LLM)"
      else:
        return "FN (supervised)" if fs == base and base == 1 else "FP (supervised)" if fs == base else "FP (All)" if base == 1 else "FN (All)"

    df["FN_FP"] = df.apply(classify, axis=1)

    if return_all:
      return df

    misclassifications = {
        "FP (All)": df[df.FN_FP == "FP (All)"].text.to_list(),
        "FP (supervised)": df[df.FN_FP == "FP (supervised)"].text.to_list(),
        "FP (LLM)": df[df.FN_FP == "FP (LLM)"].text.to_list(),
        "FN (All)": df[df.FN_FP == "FN (All)"].text.to_list(),
        "FN (supervised)": df[df.FN_FP == "FN (supervised)"].text.to_list(),
        "FN (LLM)": df[df.FN_FP == "FN (LLM)"].text.to_list()
    }

    for key, value in misclassifications.items():
      print(f"--- {key.replace('_', ' ')} count: -- {len(value)}")

    print(f"\nPrinting {n} examples of each classifier type.\n")

    for key, value in misclassifications.items():
      print(f"------ {key.replace('_', ' ').upper()} ------")
      for example in value[:n]:
        print(f"- {example}")
      print("--------")


  def RAMP_plot(self, x_col, y_col, group_col,
                pastel_colors = ['#77B5FE', '#FF6961', '#B19CD9'],
                title="", width=800, height=500, line_width=2, line_opacity=0.5,
                font_size=14, tick_size=12):
    """
    Creates a connected scatter plot with customizable font size and tick size
    """
    self.df[group_col] = self.df[group_col].astype('category')

    scatter_fig = px.line(self.df, x=x_col, y=y_col, color=group_col,
                          title=title, template='plotly_white',
                          labels={x_col: x_col, y_col: y_col, group_col: group_col},
                          markers=True,
                          color_discrete_sequence=pastel_colors)

    for group, group_df in self.df.groupby(group_col):
        min_x = group_df[x_col].min()
        max_x = group_df[x_col].max()
        min_y = group_df[group_df[x_col] == min_x][y_col].iloc[0]
        max_y = group_df[group_df[x_col] == max_x][y_col].iloc[0]
        color_index = group_df[group_col].cat.codes.unique()[0] % len(pastel_colors)
        scatter_fig.add_trace(go.Scatter(
            x=[min_x, max_x],
            y=[min_y, max_y],
            mode='lines',
            name=f'{group} - Range Line',
            line=dict(color=pastel_colors[color_index], width=line_width, dash='dash'),
            opacity=line_opacity,
            showlegend=False))

    # Update layout to include font size and tick size settings
    scatter_fig.update_layout(
        title=dict(text=title, font=dict(size=font_size)),
        xaxis=dict(title=dict(text=x_col, font=dict(size=font_size)),
                   tickfont=dict(size=tick_size)),
        yaxis=dict(title=dict(text=y_col, font=dict(size=font_size)),
                   tickfont=dict(size=tick_size)),
        legend=dict(font=dict(size=font_size)),
        width=width, height=height
    )

    scatter_fig.show()



# 1. Manual Coding (RAMP method)

This section details the results of the manual coding (1st) stage of RAMP.

## 1.1 Input

The raw dataset contains sentences from online dialogues, sampled from three sources:

1. **Reddit conversations from 27 subreddits**: This data was downloaded using the Reddit API by the authors.

2. **Twitter Customer Support data**  ([Thought Vector & Axelbrooke, 2017](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter)): This data was downloaded from (Copyright: CC BY-NC-SA 4.0).

3. **Wikipedia Talk Pages data** ([Danescu-Niculescu-Mizil et al., 2012](https://convokit.cornell.edu/documentation/wiki.html)): This data was downloaded using Cornell University's [ConvoKit](https://convokit.cornell.edu) Python package (Copyright: CC BY 4.0)

**Notes:**

> All author names and sentences have been anonymized following ethical guidelines for the study.

> As a further precaution, the sentences are shuffled and the source (e.g., Reddit, Twitter) removed from the dataframe.

In [ ]:
# Manual coded dataset final size
print(f"Full dataset size: {len(train) + len(test)}")

Full dataset size: 21982


In [ ]:
# Explanation of shared subsamples manual coding
tdf = St1Through[St1Through.Round!="one-shot"].drop(["Coder5","Coder6"],axis=1).copy()
prev_rounds = tdf[tdf.Round!="6"]
IRR_sample = tdf[tdf.Round=="6"]
IRR_texts = IRR_sample.text.to_list()
non_IRR_texts = prev_rounds.text.to_list()
crossover_texts = [t for t in IRR_texts if t in non_IRR_texts]
crossover_df = tdf[tdf.text.isin(crossover_texts)].drop_duplicates()
crossover_df.loc[:,"All_coders"] = crossover_df.Coder1 + crossover_df.Coder2 + crossover_df.Coder3 + crossover_df.Coder4
crossover_df.loc[:,"All_coders"] = crossover_df["All_coders"].apply(lambda x: 1 if x > 1 else 0)

all_rounds_len = len(tdf)
n_crossovers_all = all_rounds_len - len(tdf.text.drop_duplicates())
pct_all_cross = round((n_crossovers_all/all_rounds_len)*100,2)
IRR_len = len(IRR_sample)
n_crossovers = len(crossover_texts)
pct_cross= round((n_crossovers/IRR_len)*100,2)
n_mis_in_cross = crossover_df.All_coders.sum()
pct_mis = round((n_mis_in_cross/n_crossovers)*100,2)
print(f"""
There are {all_rounds_len} sentences across all rounds of coding.
{n_crossovers_all} ({pct_all_cross}%) sentences were shared across various rounds.

The IRR set (Round 6) contains {IRR_len} sentences.
Of these sentences, {n_crossovers} ({pct_cross}%) appeared in another round.
Of these crossover sentences, {n_mis_in_cross} ({pct_mis}%) were coded for misunderstanding.
""")

IRR_df_no_crossovers = IRR_sample[~IRR_sample.text.isin(crossover_texts)]

# Get the IRR from the final round of coding
IRR_final = tdf[tdf["Round"]=="6"]
IRR_through = tdf[tdf["Round"]!="6"]


There are 6322 sentences across all rounds of coding.
378 (5.98%) sentences were shared across various rounds.

The IRR set (Round 6) contains 1610 sentences.
Of these sentences, 174 (10.81%) appeared in another round.
Of these crossover sentences, 16.0 (9.2%) were coded for misunderstanding.



## 1.2 Throughput

In [ ]:
tds = TextDataStats(IRR_through)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 713, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 1228, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 1101, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 808, Raters: 4, Categories: [0, 1], Weights: "identity">
<irrCAC.raw.CAC Subjects: 862, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 1,713,0.95,"0.57 CI = (0.48, 0.65)",z = 13.27; p < 0.001,"0.94 CI = (0.93, 0.96)",z = 132.82; p < 0.001
1,Round 2,1228,0.97,"0.71 CI = (0.63, 0.78)",z = 18.77; p < 0.001,"0.97 CI = (0.96, 0.98)",z = 257.43; p < 0.001
2,Round 3,1101,0.97,"0.72 CI = (0.66, 0.79)",z = 21.51; p < 0.001,"0.96 CI = (0.95, 0.97)",z = 205.61; p < 0.001
3,Round 4,808,0.94,"0.78 CI = (0.73, 0.82)",z = 34.45; p < 0.001,"0.93 CI = (0.91, 0.94)",z = 104.24; p < 0.001
4,Round 5,862,0.98,"0.76 CI = (0.69, 0.83)",z = 21.29; p < 0.001,"0.98 CI = (0.97, 0.98)",z = 240.22; p < 0.001


We ended the training at Round 5, as the agreement diminishes from the previous round.

We observed that the alpha gets progressively better across the inference loop iterations.

We also saw the deceptive nature of absolute agreement. For instance, the low alpha of 0.57 in the first round has 95% agreement is because coders were generally good at recognizing *not* misunderstandings but bad at agreeing on what sentences were misunderstandings. The problem is caused by the skewed nature of the dataset (misunderstandings only 8% of data).




## 1.3 Output

The below shows the IRRs for the final round of coding. This was done blind by the coders whilst they scored the final dataset. It used the last version of the codebook (Supplementary Materials A).

In [ ]:
tds = TextDataStats(IRR_final)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 1610, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 6,1610,0.98,"0.79 CI = (0.74, 0.84)",z = 29.82; p < 0.001,"0.98 CI = (0.97, 0.98)",z = 344.58; p < 0.001


We found that the coders had good agreement (98%) with moderate inter-rater reliability (Krippendorff's Alpha  = 0.79).

### 1.3.1 Sense check manual coding output

Some sentences were scored previous to the final coding (i.e., during the throughput). To ensure that the inter-rater reliability was not biased by these sentences, we can check the statistics after removing all sentences that appeared in the throughput. This leaves only sentences had not yet scored.

In [ ]:
tds = TextDataStats(IRR_df_no_crossovers)
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 1436, Raters: 4, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round 6,1436,0.98,"0.79 CI = (0.74, 0.85)",z = 28.32; p < 0.001,"0.98 CI = (0.97, 0.98)",z = 323.78; p < 0.001


We found that the statistics remain almost identical, apart from a marginal increase in the confidence intervals for the Krippendorff's alpha.

# 2. Classifier development (RAMP method)

This stage reports the development of three classifiers on training and validation data (throughput) and their performance on the test data (output).

The throughput phase reports evaluation metrics across 21 different attempts to improve the classifiers' performance on the training data.



In [ ]:
def get_model_report(true_scores, pred_scores, display=True):
  """
  Calculate evaluation metrics.
  """
  # Load true scores and predicted scores
  # Check predictions and true scores are the same length
  print(f"true:{len(true_scores)},pred:{len(pred_scores)}")
  # Create confusion matrix
  cm = confusion_matrix(true_scores, pred_scores)
  # Fetch true/false positives and true/false negatives
  TP = cm[1, 1]
  FP = cm[0, 1]
  TN = cm[0, 0]
  # Create classification reports
  report = classification_report(true_scores, pred_scores, output_dict=True)
  try:
    # Fetch evaluation metrics
    F1_var = report['1']['f1-score']  # F1 score for class '1'
    F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
    Precision = report['1']['precision']
    Recall = report['1']['recall']
  except:
    F1_var = report['1.0']['f1-score']  # F1 score for class '1'
    F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
    Precision = report['1.0']['precision']
    Recall = report['1.0']['recall']
  # Calculate precision_recall_curve
  precision, recall, thresholds = precision_recall_curve(true_scores, pred_scores)
  # Get auea under precision recall curve
  AUC_PR = auc(recall, precision)
  # Get area under ROC
  AUC_ROC = roc_auc_score(true_scores, pred_scores)
  mcc = matthews_corrcoef(true_scores, pred_scores)
  # Print or save results
  if display:
    print(f'AUC-PR: {AUC_PR:.2f}\n')
    print(f'AUC-ROC: {AUC_ROC:.2f}\n')
    print(f'MCC: {mcc:.2f}\n')
    print(classification_report(true_scores, pred_scores))
  else:
    outdf = pd.DataFrame({"Precision":[Precision],
                          "Recall":[Recall],
                          "MCC":[mcc],
                          "AUC_PR":[AUC_PR],
                          "AUC_ROC":[AUC_ROC],
                          "F1_avg":[F1_avg],
                          "F1_var":[F1_var],
                          "TP":[TP],
                          "TN":[TN],
                          "FP":[FP],
                          "FN":[FN]})
    return outdf

def matthews_correlation_coefficient(tp, tn, fp, fn):
    numerator = (tp * tn) - (fp * fn)
    denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
    if denominator == 0:
        return 0  # Undefined MCC, return 0 as a safe default
    return numerator / denominator

## 2.1 Input

The input phase involves splitting the data into train and test data and determining the classifiers for development in the throughput.

The three classifiers are:

1. A rule-based dictionary classifier; This classifier labels a text as misunderstandings if it identifies any of a pre-defined set of words (the dictionary). We use this for binary classification. However, it can be used for producing a ratio or frequency count of the words. In this case, a ratio is pointless as the short sentences almost never contain two words relating to misunderstandings. The frequency count will mostly be 1 or 0, and therefore a binary classification. Ratios offer more information for longer texts, as these would generate more word counts.  

2. A supervised machine learning classifier: This classifier fine-tunes a BERT ([Devlin et al., 2019](https://arxiv.org/abs/1810.04805)) model using the ktrain packages. We also plot the increasing accuracy from the development stage as we explored the use of different parameters.

3. A large language model (LLM) classifier: This classifier sends a prompt to GPT-4o (version May 13, 2024) alongside the text to label. It's response is then processed into a binary classification. This is also known as zero-shot or few-shot classification, named after how many empirical examples are included in the prompt ([Brown et al., 2020](https://arxiv.org/abs/2005.14165)).

Training and test data split:

In [ ]:
full_data_size = len(train) + len(test)

train_size = len(train)
train_pct = round(100*(train_size/full_data_size))
test_size = len(test)
test_pct = round(100*(test_size/full_data_size))

print(f"""
Full data: N = {full_data_size}
Training data: n = {train_size} ({train_pct}%)
Test data: n = {test_size} ({test_pct}%)
""")


Full data: N = 21982
Training data: n = 15391 (70%)
Test data: n = 6591 (30%)     



The three classifier have different protocols and parameters that were adjusted in the throughput.

1. Rule-based: We adjusted the terms and the term type (words or word sequences) across iterations.

2. Supervised: We adjusted the validation/training split, the number of epochs, the batch size, and the learning rate.

3. Few-shot: We adjusted the prompt and empirical examples (used with the prompt.


## 2.2 Throughput

### 2.2.1 Training and developing classifiers

This sub-section provides the code used in the throughput phase of the classification development stage of RAMP.

The details for all the classifier parameters (e.g., rule-based terms, supervised hyperparameters, LLM prompts) for the different runs can be found in the `St2Through` dataframe.

#### Rule-based development

Load class for running the rule-based classifier.

In [ ]:
class Rule_based_classifier:
  """
  Function for running the rule-based classifier
  """
  def __init__(self,texts,true_scores):

    """
    Initializes the Classifier with texts and their true classification scores.
    """
    self.texts = texts
    self.true_scores = true_scores
    self.train_size = len(texts)

  # _____________________
  # Rule-based classifiers

  def add_rule_based_terms(self, terms, mode):
    """
    Adds terms for rule-based classification and sets the type of classification based on the mode.
    """
    if mode == "pattern":
      self.patterns = terms
      self.type_ = "rule-based-1"
    elif mode == "lemma":
      self.lemmas = terms
      self.type_ = "rule-based-2"

  def classify_with_spacy_pattern(self, nlp):
    """
    Classifies texts using spaCy's pattern matching for the provided patterns.
    """
    # Initiated the spacy matcher object
    matcher = Matcher(nlp.vocab)
    # Iterate over patterns and add them to the matcher
    for pattern in self.patterns:
      matcher.add(pattern["label"], [pattern["pattern"]])
    # Count patterns in the training texts and save results
    results = []
    for text in self.texts:
      doc = nlp(text) if text else nlp("notext")
      matches = matcher(doc)
      results.append(len(matches) > 0)
    self.pred_scores = results

  def classify_with_spacy_lemma(self, nlp):
    """
    Classifies texts by checking if they contain any of the specified lemmas.
    """
    # Join all words together and convert them to spacy document
    doc = nlp(" ".join(self.lemmas))
    # Get lemmas from the spacy document
    lemma_set = set(token.lemma_ for token in doc)
    # Count lemmas in training texts and save results
    results = []
    for text in self.texts:
        doc = nlp(text.lower()) if text else nlp("notext")
        text_lemmas = set(token.lemma_ for token in doc)
        results.append(bool(text_lemmas & lemma_set))
    self.pred_scores = results

  def run_classifier(self):
    """
    Determines the type of classification to use and applies it.
    """
    if self.type_ == "rule-based-1":
      self.classify_with_spacy_pattern(nlp)
    elif self.type_ == "rule-based-2":
      self.classify_with_spacy_lemma(nlp)
    else:
      raise ValueError("Invalid classifier type specified.")

  def get_model_report(self, display=True):
    """
    Calculate evaluation metrics.
    """
    # Load true scores and predicted scores
    true_scores = self.true_scores
    pred_scores = self.pred_scores
    # Check predictions and true scores are the same length
    print(f"true:{len(true_scores)},pred:{len(pred_scores)}")
    # Create confusion matrix
    cm = confusion_matrix(true_scores, pred_scores)
    # Fetch true/false positives and true/false negatives
    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    # Create classification reports
    report = classification_report(true_scores, pred_scores, output_dict=True)
    try:
      # Fetch evaluation metrics
      F1_var = report['1']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1']['precision']
      Recall = report['1']['recall']
    except:
      F1_var = report['1.0']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1.0']['precision']
      Recall = report['1.0']['recall']
    # Calculate precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(true_scores, pred_scores)
    # Get auea under precision recall curve
    AUC_PR = auc(recall, precision)
    # Get area under ROC
    AUC_ROC = roc_auc_score(true_scores, pred_scores)
    mcc = matthews_corrcoef(self.true_scores, self.pred_scores)
    # Print or save results
    if display:
      print(f'AUC-PR: {AUC_PR:.2f}\n')
      print(f'AUC-ROC: {AUC_ROC:.2f}\n')
      print(f'MCC: {mcc:.2f}\n')
      print(classification_report(true_scores, pred_scores))
    else:
      outdf = pd.DataFrame({"Precision":[Precision],
                            "Recall":[Recall],
                            "MCC":[mcc],
                            "AUC_PR":[AUC_PR],
                            "AUC_ROC":[AUC_ROC],
                            "F1_avg":[F1_avg],
                            "F1_var":[F1_var],
                            "TP":[TP],
                            "TN":[TN],
                            "FP":[FP],
                            "FN":[FN]})
      return outdf

  def return_results(self):
    return self.pred_scores

  def run_model(self, display=True):
    """
    Run the classifier and print model report.
    """
    self.run_classifier()
    self.get_model_report(display)

  def get_misclassification(self, return_all = False):
    """
    Identify misclassifications.
    """
    df = pd.DataFrame({"text":self.texts,"true":self.true_scores,
                       "pred":self.pred_scores})
    # Function to classify each row
    def classify_row(row):
      if row['true'] == 1 and row['pred'] == 1:
        return 'TP'
      elif row['true'] == 0 and row['pred'] == 1:
        return 'FP'
      elif row['true'] == 1 and row['pred'] == 0:
        return 'FN'
      elif row['true'] == 0 and row['pred'] == 0:
        return 'TN'
    df['Classification'] = df.apply(classify_row, axis=1)
    if return_all:
      return df
    else:
      return df[df["Classification"].isin(["FP","FN"])]

  def preprocess_text(self, text):
    """
    Pre-process text to identify patterns in misclassifications.
    """
    # Load stop words
    stop_words = set(stopwords.words('english'))
    # Initiate lematizer
    lemmatizer = WordNetLemmatizer()
    # Tokenize text
    words = nltk.word_tokenize(text)
    # Lemmatize and remove stop words
    cleaned_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(cleaned_text)

  def plot_misclassifications(self, df, FN_FP="FN"):
    """
    Create wordcloud for misclassifications.
    """
    # Clean text using preprocessing function.
    df['cleaned_text'] = df['text'].apply(self.preprocess_text)
    # Select fales negatives or false positives.
    if FN_FP == "FN":
      print("Word Cloud for False Negatives:")
      texts = " ".join(df[df['Classification'] == 'FN']['cleaned_text'])
    else:
      print("Word Cloud for False Positives:")
      texts = " ".join(df[df['Classification'] == 'FP']['cleaned_text'])
    # Create and print wordcloud.
    wordcloud = WordCloud(width = 800, height = 400, background_color ='white').generate(texts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

  def add_pred_scores(self,pred_scores):
    """
    Add predicted scores (only used for calculating evaluation metrics).
    """
    self.pred_scores = pred_scores



The below shows how to run the classifier and check performance on the training texts.

In [ ]:
# For exploring the terms used in the throughput.
import ast
import json
def safe_literal_eval(x):
  try:
    return ast.literal_eval(x)
  except (ValueError, SyntaxError):
    return json.loads(x)

rule_based_terms = St2Through[St2Through["Classifier"] == "rule-based"].copy()
rule_based_terms = rule_based_terms[['Classifier', 'Order','ClassiTypeStr','PatternOrLemma', 'Terms']]
rule_based_terms['Terms'] = rule_based_terms['Terms'].apply(safe_literal_eval)
rule_based_terms.head()

,Classifier,Order,ClassiTypeStr,PatternOrLemma,Terms
21,rule-based,1,rule-based-2,lemma,"[mistook, misunderstood, misread, wtf, mistake..."
22,rule-based,2,rule-based-2,lemma,"[misapprehend, interpretation, verify, baffle,..."
23,rule-based,3,rule-based-2,lemma,"[response, redress, misread, misstep, aware, r..."
24,rule-based,4,rule-based-2,lemma,"[mishandle, misjudgment, saying, approach, awa..."
25,rule-based,5,rule-based-2,lemma,"[rejection, conversation, stumped, slip, incom..."


Run classifier using the best rule-based classifier from the RAMP method throughput (loop 14).

> Note - this takes a very long time on the training data.

In [ ]:
# training texts for RAMP method
texts = train["text"].values
# misunderstanding manually coded labels for RAMP method
true_scores = train["Misunderstanding"].values
# Final terms used in Loop 13 (best performing)
terms = rule_based_terms[rule_based_terms["Order"] == 14]["Terms"].iloc[0]
# Run the classifier
RB_Loop14_train = Rule_based_classifier(texts=texts,true_scores=true_scores)
RB_Loop14_train.add_rule_based_terms(terms, "pattern")
RB_Loop14_train.run_classifier()
RB_Loop14_train.get_model_report()

#### Supervised classifier training

In [ ]:
# For classification
class supervised_classifier:
  """
  Class for running the supervised classifier
  """
  def __init__(self,texts,true_scores):

    """
    Initializes the Classifier with texts and their true classification scores.
    """
    self.texts = texts
    self.true_scores = true_scores
    self.train_size = len(texts)

  # _____________________
  # Supervised machine learning classifier

  def add_SML_params(self,
                     save_model = False,
                     # Proportion of data to use as validation data
                     validation_size=0.30,
                     batch_size=16,
                     learning_rate=2e-5,
                     epochs=4,
                     BERTmodel="bert",
                     preprocess_mode="bert",
                     maxlen=30,
                     max_features = 50000):
    self.type_ = "supervised"
    self.save_model = save_model
    self.validation_size=validation_size
    self.batch_size=batch_size
    self.learning_rate=learning_rate
    self.epochs=epochs
    self.BERTmodel=BERTmodel
    self.preprocess_mode=preprocess_mode
    self.maxlen=maxlen
    self.max_features = max_features

  def convert_misBinary(self, return_scores = False):
    output = []
    for pred in self.pred_scores:
      if "not" in pred.lower():
        output.append(0)
      else:
        output.append(1)
    if not return_scores:
      self.pred_scores = output
    else:
      return output

  def classify_with_supervised(self):
    # create vectors of texts from loaded dataset
    validation_size = self.validation_size
    preprocess_mode = self.preprocess_mode
    sentences = self.texts
    true_scores = self.true_scores

    print(f"s:{len(sentences)},ts:{len(true_scores)}, s(ts):{sum(true_scores)}")
    # split dataset into training and validation data - stratify by true scores
    trainText, validText, trainScores, validScores = train_test_split(sentences, true_scores,
                                                                      test_size=validation_size,
                                                                      random_state=10, stratify=true_scores)
    # make training dataframe
    self.MLTrain = pd.DataFrame({"text": trainText, "Misunderstanding": [int(x) for x in trainScores]})
    # make validation dataframe
    self.MLValid = pd.DataFrame({"text": validText, "Misunderstanding": [int(x) for x in validScores]})
    # get validation texts and true scores
    self.true_scores = self.MLValid["Misunderstanding"].to_list()
    self.texts = self.MLValid["text"].to_list()
    # preprocess texts and split into validation data.
    (x_train,  y_train), (x_validation, y_validation), preproc = text.texts_from_df(train_df = self.MLTrain, text_column = "text",
                                                                            label_columns = ["Misunderstanding"], val_df = self.MLValid,
                                                                            preprocess_mode=preprocess_mode, # embeddings to use
                                                                            maxlen=self.maxlen, # max number of words for a document
                                                                            max_features = self.max_features) # size of the network
    # prime the model
    model = text.text_classifier(self.BERTmodel, train_data=(x_train, y_train), preproc=preproc)
    # create the learner object and prime the classifier
    learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=self.batch_size)
    # train the model
    learner.fit_onecycle(self.learning_rate, self.epochs)
    # save trained model for predictions
    self.learner = learner
    self.preproc = preproc

  def predict_new_texts(self, save_model=False):
    """
    Run classifier on the validation data to calculate evaluation metrics.
    """
    texts_ = self.MLValid
    texts_ = texts_["text"].to_list()
    learner = self.learner
    preproc = self.preproc
    predictor = ktrain.get_predictor(learner.model, preproc)
    preds = predictor.predict(texts_)
    self.predictor = predictor
    self.pred_scores = preds
    self.convert_misBinary()

  def save_SMLmodel(self, wd = os.getcwd()):
    """
    Save the trained classifier to working directory.
    """
    predictor = self.predictor
    predictor.save(wd)

  def return_learner(self):
    return self.learner, self.preproc

  def return_MLValid(self):
    return self.MLValid
  # _____________________
  # Functions for running classifiers
  def run_classifier(self):
    """
    Run the classifier.
    """
    self.classify_with_supervised()
    self.predict_new_texts()


  def get_model_report(self, display=True):
    """
    Calculate evaluation metrics.
    """
    # Load true scores and predicted scores
    true_scores = self.true_scores
    pred_scores = self.pred_scores
    # Check predictions and true scores are the same length
    print(f"true:{len(true_scores)},pred:{len(pred_scores)}")
    # Create confusion matrix
    cm = confusion_matrix(true_scores, pred_scores)
    # Fetch true/false positives and true/false negatives
    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    # Create classification reports
    report = classification_report(true_scores, pred_scores, output_dict=True)
    try:
      # Fetch evaluation metrics
      F1_var = report['1']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1']['precision']
      Recall = report['1']['recall']
    except:
      F1_var = report['1.0']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1.0']['precision']
      Recall = report['1.0']['recall']
    # Calculate precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(true_scores, pred_scores)
    # Get auea under precision recall curve
    AUC_PR = auc(recall, precision)
    # Get area under ROC
    AUC_ROC = roc_auc_score(true_scores, pred_scores)
    mcc = matthews_corrcoef(self.true_scores, self.pred_scores)
    # Print or save results
    if display:
      print(f'AUC-PR: {AUC_PR:.2f}\n')
      print(f'AUC-ROC: {AUC_ROC:.2f}\n')
      print(f'MCC: {mcc:.2f}\n')
      print(classification_report(true_scores, pred_scores))
    else:
      outdf = pd.DataFrame({"Precision":[Precision],
                            "Recall":[Recall],
                            "MCC":[mcc],
                            "AUC_PR":[AUC_PR],
                            "AUC_ROC":[AUC_ROC],
                            "F1_avg":[F1_avg],
                            "F1_var":[F1_var],
                            "TP":[TP],
                            "TN":[TN],
                            "FP":[FP],
                            "FN":[FN]})
      return outdf

  def return_results(self):
    return self.pred_scores

  def run_model(self, display=True):
    self.run_classifier()
    self.get_model_report(display)

  def classify_with_SML(self):
    """
    Perform classification using the configured supervised machine learning predictor.
    """
    preds = self.predictor.predict(self.texts)
    self.pred_scores = [0 if "not" in pred.lower() else 1 for pred in preds]

  def add_pred_scores(self,pred_scores):
    self.pred_scores = pred_scores



The below shows how to train the supervised classifier on the training texts and check performance on the validation data.

In [ ]:
# For exploring the hyperparameters used in the throughput
supervised_hyperparam = St2Through[St2Through["Classifier"] == "supervised"]
supervised_hyperparam = supervised_hyperparam[['Classifier', 'Order','batch_size', 'epochs', 'learning_rate', 'validation_size']]
supervised_hyperparam.head()

,Classifier,Order,batch_size,epochs,learning_rate,validation_size
42,supervised,1,32.0,5.0,0.00005,0.3
43,supervised,2,16.0,5.0,0.00005,0.3
44,supervised,3,64.0,5.0,0.00004,0.3
45,supervised,4,64.0,5.0,0.00003,0.3
46,supervised,5,64.0,5.0,0.00002,0.3


Train the classifier using the best hyperparameters from the RAMP method throughput (loop 20).

In [ ]:
# Training data texts and true scores
texts = train["text"].to_list()
true_scores = train["Misunderstanding"].to_list()

# Proportion of data used for validation
validation_size=0.30
batch_size=128
learning_rate=2e-5
epochs=4

# Run the classifier
SML_Loop20 = supervised_classifier(texts=texts,true_scores=true_scores)
SML_Loop20.add_SML_params(validation_size=validation_size,
                          batch_size=batch_size,
                          learning_rate=learning_rate,
                          epochs=epochs)

SML_Loop20.run_model()

#### LLM development

In [ ]:
class LLM_classifier:
  """
  Class for running the LLM classifier.
  """
  def __init__(self,texts,true_scores):

    """
    Initializes the Classifier with texts and their true classification scores.
    """
    self.texts = texts
    self.true_scores = true_scores
    self.train_size = len(texts)

  # LLMclassifier
  def add_prompt_param(self,GPTmodel,role,prompt,suffix,examples=""):
    """
    Add prompt to class.
    """
    self.GPTmodel = GPTmodel
    # Join prompt and examples.
    self.prompt = "\n".join([prompt,examples,suffix])
    self.role = role
    self.cost = 0
    self.total_tokens = 0

  def gptActualCost(self, response):
    """
    Calculates the gpt cost.
    """
    engine = self.GPTmodel
    total_tokens=response.usage.total_tokens
    total_tokens_1M_units = total_tokens/1000000
    if engine=='gpt-4o-2024-05-13':
        cost=total_tokens_1M_units*7
    elif engine=='gpt-4o-mini':
        cost=total_tokens_1M_units*0.15
    else:
        print('getCost error: engine not found')
        return
    return cost, total_tokens

  def get_llm_response(self,messages,temperature=0, max_tokens = 100, max_attempts = 3):
    '''
    Function that takes messages format for ChatGPT input and returns the response text.
    '''
    GPTmodel = self.GPTmodel
    for attempt in range(0, max_attempts):
      try:
        #. request timeout ADD IN
        response = openai.chat.completions.create(model=GPTmodel, messages = messages, temperature=temperature, max_tokens=max_tokens)
        response_text = response.choices[0].message.content
        self.LLMScores.append(response_text)
        response_cost, token_count = self.gptActualCost(response)
        self.cost += response_cost
        self.total_tokens += token_count
        break  # If analysis was successful, break out of the retry loop
      except Exception as e:
        print(f"Error processing text on attempt {attempt+1}: {e}")
        if attempt + 1 == max_attempts:
          print(f"Skipping text after {max_attempts} failed attempts.")
          response_text
    return response_text


  def define_messages(self, text_to_classify):
    '''
    Function for creating a basic messages format from a prompt, a role, and a text to classify (all strings)
    '''
    prompt = self.prompt
    role = self.role
    prompt = prompt.format(text_to_classify)
    messages = [{'role': 'system', 'content': role},
                {'role': 'user', 'content' : prompt}]
    return messages

  def convert_llm_scores_binary(self, return_scores = False):
    '''
    Function for converting a string "Yes" or "No" into binary format - used for the clarification requests
    '''
    llm_scores = self.pred_scores
    new_scores = []
    for s in llm_scores:
      if "yes" in s.lower():
        new_scores.append(1)
      else:
        new_scores.append(0)
    if not return_scores:
      self.pred_scores = new_scores
    else:
      return new_scores

  def classify_with_fewshot(self,  max_tokens = 100, max_attempts = 3, temperature = 0):
    '''
    Function for running a prompt over a series of texts (expects a list)
    '''

    prompt = self.prompt
    role = self.role
    input_texts = self.texts
    GPTmodel = self.GPTmodel
    scores = []
    for txt in tqdm(input_texts):
      message = self.define_messages(txt)
      try:
        response = self.get_llm_response(message,temperature=temperature,
                                         max_tokens=max_tokens, max_attempts=max_attempts)
      except:
        response = "Error in response"
      scores.append(response)
    self.pred_scores = scores
    self.convert_llm_scores_binary()


  # _____________________
  # Functions for running classifiers
  def run_classifier(self):
    """
    Determines the type of classification to use and applies it.
    """
    self.classify_with_fewshot()
    cost = self.cost
    total_tokens = self.total_tokens
    avg_tokens = self.total_tokens / self.train_size
    print(f"This run cost {cost:.2f}$ for {total_tokens} tokens. Average tokens: {avg_tokens:.2f}")

  def get_model_report(self, display=True):
    """
    Calculate evaluation metrics.
    """
    # Load true scores and predicted scores
    true_scores = self.true_scores
    pred_scores = self.pred_scores
    # Check predictions and true scores are the same length
    print(f"true:{len(true_scores)},pred:{len(pred_scores)}")
    # Create confusion matrix
    cm = confusion_matrix(true_scores, pred_scores)
    # Fetch true/false positives and true/false negatives
    TP = cm[1, 1]
    FP = cm[0, 1]
    TN = cm[0, 0]
    # Create classification reports
    report = classification_report(true_scores, pred_scores, output_dict=True)
    try:
      # Fetch evaluation metrics
      F1_var = report['1']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1']['precision']
      Recall = report['1']['recall']
    except:
      F1_var = report['1.0']['f1-score']  # F1 score for class '1'
      F1_avg = report['weighted avg']['f1-score']  # Weighted average F1 score
      Precision = report['1.0']['precision']
      Recall = report['1.0']['recall']
    # Calculate precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(true_scores, pred_scores)
    # Get auea under precision recall curve
    AUC_PR = auc(recall, precision)
    # Get area under ROC
    AUC_ROC = roc_auc_score(true_scores, pred_scores)
    mcc = matthews_corrcoef(self.true_scores, self.pred_scores)
    # Print or save results
    if display:
      print(f'AUC-PR: {AUC_PR:.2f}\n')
      print(f'AUC-ROC: {AUC_ROC:.2f}\n')
      print(f'MCC: {mcc:.2f}\n')
      print(classification_report(true_scores, pred_scores))
    else:
      outdf = pd.DataFrame({"Precision":[Precision],
                            "Recall":[Recall],
                            "MCC":[mcc],
                            "AUC_PR":[AUC_PR],
                            "AUC_ROC":[AUC_ROC],
                            "F1_avg":[F1_avg],
                            "F1_var":[F1_var],
                            "TP":[TP],
                            "TN":[TN],
                            "FP":[FP],
                            "FN":[FN]})
      return outdf

  def run_model(self, display=True):
    """
    Run the classifier and print model report.
    """
    self.run_classifier()
    self.get_model_report(display)

  def get_misclassification(self, return_all = False):
    df = pd.DataFrame({"text":self.texts,"true":self.true_scores,
                       "pred":self.pred_scores})
    # Function to classify each row
    def classify_row(row):
      if row['true'] == 1 and row['pred'] == 1:
        return 'TP'
      elif row['true'] == 0 and row['pred'] == 1:
        return 'FP'
      elif row['true'] == 1 and row['pred'] == 0:
        return 'FN'
      elif row['true'] == 0 and row['pred'] == 0:
        return 'TN'
    df['Classification'] = df.apply(classify_row, axis=1)
    if return_all:
      return df
    else:
      return df[df["Classification"].isin(["FP","FN"])]

  def preprocess_text(self, text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = nltk.word_tokenize(text)
    cleaned_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(cleaned_text)

  def plot_misclassifications(self, df, FN_FP="FN"):
    df['cleaned_text'] = df['text'].apply(self.preprocess_text)
    if FN_FP == "FN":
      print("Word Cloud for False Negatives:")
      texts = " ".join(df[df['Classification'] == 'FN']['cleaned_text'])
    else:
      print("Word Cloud for False Positives:")
      texts = " ".join(df[df['Classification'] == 'FP']['cleaned_text'])
    wordcloud = WordCloud(width = 800, height = 400, background_color ='white').generate(texts)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

  def return_LLMscores(self):
    # For returning the raw LLM output.
    return self.LLMScores

  def add_pred_scores(self,pred_scores):
    self.pred_scores = pred_scores

  def return_results(self):
    return self.pred_scores

The below shows how to run the classifier and check performance on the training texts.

In [ ]:
# For exploring the prompts and examples used in the throughput
LLM_param = St2Through[St2Through["Classifier"] == "few-shot"]
LLM_param = LLM_param[['Classifier', 'Order','role', 'prompt']]
LLM_param.head()

,Classifier,Order,role,prompt
0,few-shot,1,\nYou are a research assistant tasked with ide...,\n\n\nDoes the below sentence contain a misund...
1,few-shot,2,\nYou are a research assistant tasked with ide...,\n\n\nDoes the below sentence indicate a possi...
2,few-shot,3,\nYou are a research assistant tasked with ide...,\n\n\nDoes the below sentence indicate a possi...
3,few-shot,4,\nYou are a research assistant tasked with ide...,\n\n\nDoes the below sentence indicate a possi...
4,few-shot,5,\nYou are a research assistant tasked with ide...,\n*Your role*\nYou are a research assistant ta...


Run classifier on training data for best prompt (Loop 13)

In [ ]:
# Training data texts and true scores
texts = train["text"].to_list()
true_scores = train["Misunderstanding"].to_list()

# GPT model
GPTmodel = "gpt-4o-2024-05-13"
# fetch role from the LLM_param dataframe (loop 13)
role = LLM_param[LLM_param["Order"] == 13]["role"].iloc[0]
# fetch prompt from the LLM_param dataframe (loop 13)
prompt = LLM_param[LLM_param["Order"] == 13]["prompt"].iloc[0]

# Run classifier
FS_Loop13 = LLM_classifier(texts=texts,true_scores=true_scores)
FS_Loop13.add_prompt_param(GPTmodel=GPTmodel,role=role,prompt=prompt)
FS_Loop13.run_model()

### 2.2.2 Results

These two plots show the classifier performance on the Matthews Correlation Coefficient (MCC) and the Weighted F1 for the classifiers.

Each point indicates a change in the input parameters of the classifier. For the rule-based classifier, this was adding and altering the words. For the supervised classifier, this was altering the hyper-parameters. For the few-shot classifier, this was changing the prompt.

In [ ]:
target_cols = ["Order","Classifier","F1_avg","F1_var","AUC_PR", "AUC_ROC","Precision","Recall","FP","FN","TP","TN"]
ThroughRes = St2Through[target_cols].round(2).sort_values("AUC_PR", ascending = False)
ThroughRes["Matthews Correlation Coefficient"] = ThroughRes.apply(lambda row: matthews_correlation_coefficient(row["TP"],row["TN"],row["FP"],row["FN"]), axis=1)
ThroughRes = ThroughRes.rename(columns={"F1_avg": "Weighted F1"})
tdf = ThroughRes.sort_values(by=["Order","Classifier"])

In [ ]:
tdf = tdf.replace({"few-shot":"LLM"})
tds = TextDataStats(tdf)
tds.RAMP_plot("Order", "Matthews Correlation Coefficient","Classifier", width=1100,height=700, font_size = 20, tick_size=17)

In [ ]:
tds.RAMP_plot("Order", "Weighted F1","Classifier", width=1100,height=700)

The below shows the best classifiers from the throughput section, calculated on validation data (i.e., proportion of the training data).

In [ ]:
# Calculate MCC
St2Through["MCC"] = St2Through.apply(lambda row: matthews_correlation_coefficient(row["TP"],row["TN"],row["FP"],row["FN"]), axis=1)

In [ ]:
# Best rule-based classifier (loop 14)
ruleBased_best = St2Through[["Order","Classifier","F1_avg","AUC_PR","MCC", "Precision","Recall","PatternOrLemma","train_size","nTerms"]].rename(columns={"train_size":"validation_N"})
ruleBased_best[ruleBased_best["Classifier"]=="rule-based"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,PatternOrLemma,validation_N,nTerms
34,14,rule-based,0.87,0.32,0.22,0.33,0.24,pattern,14728,10.0


In [ ]:
# Best supervised classifier and parameters (loop 20)
supervised_best = St2Through[["Order","Classifier","F1_avg","AUC_PR", "MCC","Precision","Recall","validation_size","epochs","learning_rate","batch_size"]].rename(columns={"validation_size":"validation_proportion"})
supervised_best[supervised_best["Classifier"]=="supervised"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,validation_size,epochs,learning_rate,batch_size
61,20,supervised,0.94,0.7,0.65,0.74,0.62,0.3,4.0,0.0,128.0


In [ ]:
# Best LLM classifiers (loop 13)
LLM_best = St2Through[["Order","Classifier","F1_avg","AUC_PR", "MCC","Precision","Recall", "train_size","GPTmodel"]].rename(columns={"train_size":"validation_N"})
LLM_best[LLM_best["Classifier"]=="few-shot"].sort_values("MCC", ascending = False).head(1).round(2)

,Order,Classifier,F1_avg,AUC_PR,MCC,Precision,Recall,validation_N,GPTmodel
12,13,few-shot,0.91,0.58,0.51,0.54,0.59,1000,gpt-4-turbo


## 2.3 Output

The below details the evaluation of the three classifiers on the test data, withheld from the throughput stage. We fetch the true scores from the test data:

In [ ]:
true_scores = out.Misunderstanding.to_list()

### 2.3.1 Rule-based classifier

Set the terms (from throughput loop 14):

In [ ]:
terms = [{"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["what","why"]}},{"OP":"*","POS":"AUX"},{"POS":"VERB","OP":"*"},{"IS_PUNCT":True,"OP":"?"}]},
          {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"what"},{"LOWER":"do"},{"LOWER":"you"},{"LOWER":"mean"},{"IS_PUNCT":True,"OP":"?"}]},
            {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"could"},{"LOWER":"you"},{"LEMMA":{"IN":["elaborate","expand","explain"]}},{"OP":"?","IS_PUNCT":True}]},
             {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"i"},{"LOWER":"don't"},{"LEMMA":"understand"}]},
              {"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["sorry","pardon","excuse"]}},{"LOWER":"me"},{"LOWER":"could"},{"LOWER":"you"},{"LOWER":"repeat"},{"OP":"?","IS_PUNCT":True}]},
               {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"are"},{"LOWER":"you"},{"LOWER":"saying"},{"IS_ALPHA":True,"OP":"+"},{"OP":"?","IS_PUNCT":True}]},
                {"label":"MISUNDERSTANDING","pattern":[{"LEMMA":{"IN":["misinterpret","misunderstand","misconstrue"]}},{"POS":"ADP"},{"IS_ALPHA":True,"OP":"+"}]},
                 {"label":"MISUNDERSTANDING","pattern":[{"LOWER":{"IN":["did","do","does"]}},{"LOWER":"you"},{"LEMMA":"mean"},{"OP":"?","IS_PUNCT":True}]},
                  {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"let's"},{"LOWER":"talk"},{"LOWER":"about"},{"IS_ALPHA":True,"OP":"+"}]},
                   {"label":"MISUNDERSTANDING","pattern":[{"LOWER":"to"},{"LOWER":"clarify"},{"OP":"?","IS_PUNCT":True}]}]

Run the classifier (unhash to replicate - takes a long time).

In [ ]:
#RB_Loop14_train = Rule_based_classifier(texts=texts,true_scores=true_scores)
#RB_Loop14_train.add_rule_based_terms(terms, "pattern")
#RB_Loop14_train.run_classifier()
#RB_Loop14_train.get_model_report()

Results (saved for convenience)

In [ ]:
pred_scores = out["rule-based"].to_list()
get_model_report(true_scores, pred_scores)

true:6420,pred:6420
AUC-PR: 0.31

AUC-ROC: 0.61

MCC: 0.22

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      5909
           1       0.29      0.27      0.28       511

    accuracy                           0.89      6420
   macro avg       0.62      0.61      0.61      6420
weighted avg       0.89      0.89      0.89      6420



### 2.3.2 Supervised classifier

The classifier was trained during the throughput (loop 20) so to run it on the test data requires loading it into the runtime.

In [ ]:
# Load trained supervised model (downloaded in section 0.2)
predictor = ktrain.load_predictor('supervised_model')

Test the classifier has been correctly loaded

In [ ]:
# Misunderstanding example
predictor.predict("I don't understand your point.")

'Misunderstanding'

In [ ]:
# Not misunderstanding example
predictor.predict("Cats are not the same as dogs.")

'not_Misunderstanding'

Run the classifier (unhash to replicate).

In [ ]:
#smlClassifier = supervised_classifier(texts, true_scores)
#smlClassifier.add_SML_classifier(predictor)
#smlClassifier.classify_with_SML()
#smlClassifier.get_model_report()

Results saved for convenience:

In [ ]:
true_scores = out.Misunderstanding.to_list()
pred_scores = out["supervised"].to_list()
get_model_report(true_scores, pred_scores)

true:6420,pred:6420
AUC-PR: 0.73

AUC-ROC: 0.88

MCC: 0.69

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      5909
           1       0.65      0.79      0.71       511

    accuracy                           0.95      6420
   macro avg       0.81      0.88      0.84      6420
weighted avg       0.95      0.95      0.95      6420



### 2.3.3 LLM classifier


Load the role and prompt (loop 13):

In [ ]:
role = """
*Role* You are a research assistant tasked with identifying whether a sentence indicates a misunderstanding.
*Misunderstanding definition* A misunderstanding occurs during dialogue when one participant has an incorrect understanding of another’s perspective.
"""
prompt = """
There are two categories of misunderstanding:
1. “Direct” misunderstandings. These occur when a participant evidences a misunderstanding of another participant’s point.
2. “Felt” misunderstandings. These occur when a participant feels their previous turn was misunderstood by another participant.
This is a non-exhaustive list of possible sentences indicating misunderstanding.
1. Explicit statement: The sentence explicitly indicates the speaker doesn't understand another’s perspective (e.g., "I don't get what you're trying to say about the dog")
2. Clarification question: The question seeks to clarify the other’s perspective (e.g., "What do you mean?")
3. Request for confirmation: A question that seeks confirmation on the other’s understanding of the speaker’s previous turn(e.g., "You really think that I meant all dogs?")
4. Correction of Other: Correcting another speaker’s misunderstanding of the present speaker’s previous turn(e.g., "You've misunderstood my point", “You don’t get it.”)
5. Clarification or apology about speaker's intentions: Clarifying the meaning of what the speaker previously said (e.g., "Sorry, I meant to say X")
6. Misunderstanding due to lack of response (e.g., "Why did you change the subject?")
7. Editing a message at a later time: This is when a speaker in text-based dialogue comes back to edit their comment after the fact (e.g., "EDIT": That's what I said)
Here are some examples of sentences indicating misunderstandings:
- Jane, that article was what I was talking about.
- Why not go further? - Do you think that was ok?
- I apologise for saying that, but I meant the other stuff.
- @John But when? - @John Please tell me why I've been stuck here for so long.
- What drove that thought? - I actually said "sure thing".
- You serious?
- I'm not sure what I could have done differently.
TASK:
Does the below sentence indicate a possible misunderstanding?
Only respond with "Yes" or "No"
Sentence: {}
Response:"""

Run the classifier. Unhash to replicate; results are likely to be marginally different due to the LLM's stochastic behavior.

In [ ]:
## GPT model
#GPTmodel = "gpt-4o-2024-05-13"
## Run classifier
#LLmclass = LLM_classifier(texts=texts,true_scores=true_scores)
#LLmclass.add_prompt_param(GPTmodel=GPTmodel,role=role,prompt=prompt)
#LLmclass.run_model()

In [ ]:
true_scores = out.Misunderstanding.to_list()
pred_scores = out["few-shot"].to_list()
get_model_report(true_scores, pred_scores)

true:6420,pred:6420
AUC-PR: 0.56

AUC-ROC: 0.80

MCC: 0.47

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      5909
           1       0.39      0.69      0.50       511

    accuracy                           0.89      6420
   macro avg       0.68      0.80      0.72      6420
weighted avg       0.93      0.89      0.90      6420



# 3. Integrative evaluation (RAMP method)

This section looks at disagreements and misclassifications in order to inform the final stage of RAMP. These are used to infer surprising findings from which to identify potential problems of construct and concept validity.

This analysis is qualitative and is informed by the below disagreements and misclassifications

## 3.1 Inter-rater discrepancies

In [ ]:
# Get sample of disagreements
tds = TextDataStats(IRR_final)
tds.get_disagreements(25)

----------
In any case, everyone has different things they find satisfying to do on Wikipedia; why don't you spend time on things that give you pleasure?
----------
@Ask_Spectrum I've gave your company enough of my patience ive had enough, you just lost a customer!.
----------
Update: as a few have pointed out, the term racist was a poor choice of words.
----------
this is amc, but i feel you
----------
Well, I'm not talking about Western Sahara specifically.
----------
We wouldn't be able to comment further than what was discussed yesterday, until Omniserve come back.
----------
The rest not so rightÔ£ø√º√≤√ë thanks for correcting me!
----------
The article is actually a lot better and resourceful than it originally appeared but I believe my edits have improved it, even if I picked up a few horses in Jutland rather than Jutland horse and probably needed minor copyedits.
----------
Why go from zero to 100 today?
----------
I mean is there proof for that third one?
----------
You seem t

## 3.2 Supervised misclassifications

In [ ]:
out2 = out.copy()
out2 = out2.rename(columns={"Misunderstanding":"Manual", "few-shot":"LLM"})

In [ ]:
tds = TextDataStats(out2)
misdf = tds.get_misclassifications(return_all = True)

In [ ]:
# For printing rule-based misclassifications - these are fairly arbitrary
for i in misdf[misdf["Manual"]==1][misdf["rule-based"]==0].sample(10).text.values:
  print(i)

btw I'm not Ronald!
hehe, whoops!
No news is good news?
@Company_Handle For the type of issue reported I thought I may have at least been contacted for some information.
Do you steal from seniors too or just kids?
Am I missing something?
Isn't this to show pics we've taken??
We don't think it is.
Like John hasn't exhibited a consistent attitude conducive to collaboration?
Oh, I see.


<ipython-input-17-3b1cc0282d4c>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in misdf[misdf["Manual"]==1][misdf["rule-based"]==0].sample(10).text.values:


In [ ]:
# For supervised and few shot classifiers
tds = TextDataStats(out2)
tds.get_misclassifications(n=25)

--- FP (All) count: -- 53
--- FP (supervised) count: -- 111
--- FP (LLM) count: -- 436
--- FN (All) count: -- 111
--- FN (supervised) count: -- 53
--- FN (LLM) count: -- 0

Printing 25 examples of each classifier type.

------ FP (ALL) ------
- Sure, do you want me to?
- This is completely unacceptable and to worsen it is that you lot do not respond quick enough.
- I am very patient as to what-will-happen-next, although I don't seem to manage it with spur-of-the-moment replies (oops).
- I'm not really committed to extensive rewriting as it's only wiki and can be edited anyway.
- I take that back.
- hahahahaha my apologies
- How can you fame a Wikipedian?
- @Company_Handle @Company_Handle What type of ticket have you bought?
- Again, Industry Canada is a reliable source, even if not the preferred one, and those numbers are much better than no numbers at all.
- Anything you can do to make him believe that I mean no ill will would be very helpful.
- For one i didn't even know the person \

### 3.2.1 Using LIME to explore BERT classifier

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("It's me, again")

In [ ]:
predictor.explain("Mark, the experience you are describing is something we'd never do.")

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("Not sure what that is")

In [ ]:
# For assessing the supervised classifier FN and FP
predictor.explain("But... They do.")

# 4. Non-iterative method results

To compare the RAMP approach, we had two coders separately score the entire dataset using the original version of the manual codebook, prior to the changes integrated during the inference loops. We then use this data to test three classifiers.

The rule-based classifier uses the terms from loop 1 of RAMP method troughput (prior to any iterations), the supervised classifier was trained on 70% of the data (30% of the data used for validation) using the hyperparameters from loop 1 of RAMP method troughput, and the few-shot classifier uses the prompt from loop 1 of RAMP method troughput.

## 4.1 Manual coding

In [ ]:
nMis_oneShot = train_test.Misunderstanding_OneShot.sum()
pct_misOneShot = round(100*(nMis_oneShot/len(train_test)),2)
print(f"Identified {nMis_oneShot} misunderstandings in the one-shot, accounting for {pct_misOneShot}")

Identified 592.0 misunderstandings in the one-shot, accounting for 2.69


In [ ]:
one_shot_MC = St1Through[St1Through.Round == "one-shot"]
tds = TextDataStats(one_shot_MC, IRR_columns = ["Coder5","Coder6"])
tds.IRRreport().round(2)

<irrCAC.raw.CAC Subjects: 2000, Raters: 2, Categories: [0, 1], Weights: "identity">


,Round,Sample size,Agreement,K's Alpha,K's Alpha significance,Gwet's AC1,Gwet's AC1 significance
0,Round one-shot,2000,0.95,"0.28 CI = (0.18, 0.39)",z = 5.41; p < 0.001,"0.95 CI = (0.94, 0.96)",z = 179.07; p < 0.001


We found that the inter-rater reliability is, on the whole, worse for the one-shot group than any of the iterations of RAMP (including Round 1).

### 4.1.1 Compare RAMP and non-iterative manual coding

The below examines the differences between the RAMP and non-iterative  method manual coding so that we can see where the coders differed in their codes of misunderstanding.

In [ ]:
# Assuming true scores are the RAMP scores
data_to_compare = train_test[["Misunderstanding", "Misunderstanding_OneShot"]]
print(classification_report(data_to_compare["Misunderstanding"], data_to_compare["Misunderstanding_OneShot"]))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97     20267
           1       0.72      0.25      0.37      1715

    accuracy                           0.93     21982
   macro avg       0.83      0.62      0.67     21982
weighted avg       0.92      0.93      0.92     21982



In [ ]:
cm_compare = confusion_matrix(data_to_compare["Misunderstanding"], data_to_compare["Misunderstanding_OneShot"])
TP = cm_compare[1, 1]
FN = cm_compare[1, 0]
FP = cm_compare[0, 1]
TN = cm_compare[0, 0]
total_shared = TP + TN
pct_shared = round(100*(total_shared/len(data_to_compare)),2)
print(f"One-shot and RAMP coding share {total_shared} ({pct_shared}%) codes.\n")
total_disagree = FP + FN
pct_disagree = round(100*(total_disagree/len(data_to_compare)),2)
print(f"One-shot and RAMP coding disagree on {total_disagree} ({pct_disagree}%) codes.\n")
pct_disagree_FN = round(100*(FN/total_disagree),2)
print(f"Of the disagreements {FN} ({pct_disagree_FN}%) were cases where one-shot coders DID NOT score for misunderstanding and RAMP coders didn't.\n")
pct_disagree_FP = round(100*(FP/total_disagree),2)
print(f"Of the disagreements {FP} ({pct_disagree_FP}%) were cases where one-shot coders DID SCORE for misunderstanding and RAMP coders did.\n")
print(f"Instances of misunderstandings identified by positive coders (true positives): {TP}")

One-shot and RAMP coding share 20529 (93.39%) codes.

One-shot and RAMP coding disagree on 1453 (6.61%) codes.

Of the disagreements 1288 (88.64%) were cases where one-shot coders DID NOT score for misunderstanding and RAMP coders didn't.

Of the disagreements 165 (11.36%) were cases where one-shot coders DID SCORE for misunderstanding and RAMP coders did.

Instances of misunderstandings identified by positive coders (true positives): 427


## 4.2 Classifier development

### 4.2.1 Training and developing classifiers

The below provides the code for training the supervised classifier. The rule-based and LLM classifier were only tested as the terms and prompt (respectively) were already created.

In [ ]:
# Training data texts and true scores for non-iterative method
texts_NonIterative = train_NonIterative["text"].values
true_scores_NonIterative  = train_NonIterative["Misunderstanding_OneShot"].values

# Set hyperparameters and validation size
validation_size=0.30 # Proportion of data used for validation
batch_size= 32
learning_rate=5e-5
epochs=5

# Run the classifier
SML_NonIterative = supervised_classifier(texts=texts_NonIterative,true_scores=true_scores_NonIterative)
SML_NonIterative.add_SML_params(validation_size=validation_size,
                                batch_size=batch_size,
                                learning_rate=learning_rate,
                                epochs=epochs)
# << Unhash below to train new model >>

#SML_NonIterative.run_model()

We can also see the Rule-based classifier terms:

In [ ]:
One_shot_terms = St2Through[St2Through["Classifier"] == "rule-based"]
One_shot_terms = One_shot_terms[One_shot_terms["Order"]==1]
One_shot_terms.Terms.values[0]

'["mistook","misunderstood","misread","wtf","mistake","response","stumped","uncertain","restate","revise"]'

And the prompt used for the LLM classifier:

In [ ]:
One_shot_LLM = St2Through[St2Through["Classifier"] == "few-shot"]
One_shot_LLM = One_shot_LLM[One_shot_LLM["Order"]==1]
print(One_shot_LLM.role.values[0])
print(One_shot_LLM.prompt.values[0])


You are a research assistant tasked with identifying whether a sentence indicates a misunderstanding




Does the below sentence contain a misunderstanding?
Only respond with "Yes" or "No"
Sentence: {} 
Response:


### 4.2.2 Output

The below prints the classification reports and evaluation metrics for the non-iterative test data.

In [ ]:
# Fetch test data misunderstanding manual coding scores
true_values_NonIterative = out_NonIterative["Misunderstanding_OneShot"]

In [ ]:
print(f"Test size for one-shot group: {len(out_NonIterative)}")

Test size for one-shot group: 6595


Rule-based classification report:

In [ ]:
rule_based_NonIterative=  out_NonIterative["Rule_based_preds"]
get_model_report(true_values_NonIterative, rule_based_NonIterative)

true:6595,pred:6595
AUC-PR: 0.06

AUC-ROC: 0.51

MCC: 0.03

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      6417
         1.0       0.07      0.02      0.03       178

    accuracy                           0.97      6595
   macro avg       0.52      0.51      0.51      6595
weighted avg       0.95      0.97      0.96      6595



Supervised classification report

In [ ]:
supervised_NonIterative =  out_NonIterative["SML_preds"]
get_model_report(true_values_NonIterative, supervised_NonIterative)

true:6595,pred:6595
AUC-PR: 0.39

AUC-ROC: 0.64

MCC: 0.35

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99      6417
         1.0       0.46      0.30      0.36       178

    accuracy                           0.97      6595
   macro avg       0.72      0.64      0.67      6595
weighted avg       0.97      0.97      0.97      6595



LLM classification report:

In [ ]:
LLM_NonIterative =  out_NonIterative["LLM_scores"]
get_model_report(true_values_NonIterative, LLM_NonIterative)

true:6595,pred:6595
AUC-PR: 0.16

AUC-ROC: 0.47

MCC: -0.02

              precision    recall  f1-score   support

         0.0       0.97      0.66      0.79      6417
         1.0       0.02      0.28      0.04       178

    accuracy                           0.65      6595
   macro avg       0.50      0.47      0.41      6595
weighted avg       0.94      0.65      0.77      6595



# 4. Conclusions

For the RAMP method, the manual coding stage found acceptable inter-rater reliability (Krippendorff's Alpha = 0.79) following 5 training rounds; the classifier development stage found that the supervised  classifier is much better than either the LLM and rule-based classifiers. The supervised classifier's performance is acceptable (MCC = 0.69) with room for improvement. For the RAMP method's integrative evaluation stage, we found that the false negatives are missing some key clarification questions (e.g., "So what about everyone else?"). We also found that the classifiers are picking up on "new information" questions, not directed at another's perspeective (e.g., "Are you having this issue with any other channels?"). The misclassifications indicate that the classifier is generally struggling with edge cases more than explicit cases.

When comparing the RAMP method with the non-iterative method, we found that the RAMP method performs better in both the manual coding and classifier development stages. We also found that the non-iterative manual coders found fewer instances of misunderstanding but, those they did find, were mostly identified by the RAMP coders.



# 5. References


> Brown, T., Mann, B., Ryder, N., Subbiah, M., Kaplan, J. D., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., Agarwal, S., Herbert-Voss, A., Krueger, G., Henighan, T., Child, R., Ramesh, A., Ziegler, D., Wu, J., Winter, C., … Amodei, D. (2020). Language models are few-shot learners. Advances in Neural Information Processing Systems, 33, 1877–1901. https://proceedings.neurips.cc/paper/2020/hash/1457c0d6bfcb4967418bfb8ac142f64a-Abstract.html

> Danescu-Niculescu-Mizil, C., Lee, L., Pang, B., & Kleinberg, J. (2012). Echoes of power: Language effects and power differences in social interaction. Proceedings of the 21st International Conference on World Wide Web, 699–708. https://doi.org/10.1145/2187836.2187931

> Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of deep bidirectional transformers for language understanding. Proceedings of the 2019 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long and Short Papers), 4171–4186. https://doi.org/10.18653/v1/N19-1423

> Honnibal, M., Montani, I., Van Landeghem, S., & Boyd, A. (2022). SpaCy: Industrial-Strength Natural Language Processing in Python. Explosion.

> Korobov, M. (Presenter). (2017). Explaining behavior of Machine Learning models with eli5 library. EuroPython.

> Korobov, M., & Lopuhin, K. (2024). eli5: Debug machine learning classifiers and explain their predictions (0.13.0) [Python; OS Independent]. https://github.com/eli5-org/eli5

> Krippendorff, K. (1970). Estimating the reliability, systematic error and random error of interval data. Educational and Psychological Measurement, 30(1), 61–70. https://doi.org/10.1177/001316447003000105

> Maiya, A. S. (2022). ktrain: A low-code library for augmented machine learning (arXiv:2004.10703). arXiv. https://doi.org/10.48550/arXiv.2004.10703

> OpenAI, Achiam, J., Adler, S., Agarwal, S., Ahmad, L., Akkaya, I., Aleman, F. L., Almeida, D., Altenschmidt, J., Altman, S., Anadkat, S., Avila, R., Babuschkin, I., Balaji, S., Balcom, V., Baltescu, P., Bao, H., Bavarian, M., Belgum, J., … Zoph, B. (2024). GPT-4 Technical Report (arXiv:2303.08774). arXiv. https://doi.org/10.48550/arXiv.2303.08774

> Pennebaker, J. W., Francis, M. E., & Booth, R. J. (2001). Linguistic Inquiry and Word Count: LIWC. Mahway: Lawrence Erlbaum Associates, 71(2001).

> Ribeiro, M. T., Singh, S., & Guestrin, C. (2016). ‘Why should I trust you?’: Explaining the predictions of any classifier (arXiv:1602.04938). arXiv. https://doi.org/10.48550/arXiv.1602.04938

> Shivan, B., & Chaitanya, A. (2024). textstat: Calculate statistical features from text (0.7.3) [Python]. https://github.com/shivam5992/textstat

